## Step 1
We first import all our libraries and module utilities

In [33]:
import torch
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn

## Step 2
Load the MNIST dataset from torchvision

In [7]:
train_data = datasets.MNIST(root='./',
                           train=True,
                           download=True,
                           transform=transforms.ToTensor())
test_data = datasets.MNIST(root='./',
                          train=False,
                          download=True,
                          transform=transforms.ToTensor())

100.1%

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


113.5%

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


100.4%

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


180.4%

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/Users/wasch/.local/share/virtualenvs/nn-mnist-classification-nKJ1CUJu/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [50]:
# train data consists of 60,000 examples of digit images
example = train_data[0]
ex_img = example[0] # pull one example and get the it's image tensor
print(ex_img.size()) # check the size of our inputs
ex_img = transforms.ToPILImage()(ex_img) # convert the image tensor into a PIL object
display(ex_img) # view the image

torch.Size([1, 28, 28])


## Step 3
Split our train data into our training and validation data

In [16]:
train_data, valid_data = torch.utils.data.random_split(train_data, [50000, 10000])

In [17]:
train_loader = DataLoader(train_data,
                          batch_size=20,
                          shuffle=True
                         )
valid_loader = DataLoader(valid_data,
                          batch_size=20,
                          shuffle=True
                         )
test_loader = DataLoader(test_data,
                         batch_size=20,
                         shuffle=True
                        )

## Step 4
Create our digit classification NN model

In [25]:
class DigitClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=1,
                      out_channels=3,
                      kernel_size=3), # 3 x 26 x 26
            nn.ReLU(), # apply activations: ReLU since we have > 0 classes
            nn.Flatten(), # flatten our input before passing it through our fully connected layer
            nn.Linear(3*26*26, 10)
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x

## Step 5
Instantiate our criterion and optimizers and define our trainer

In [35]:
lr = 0.0001
epochs = 1

dc_cnn = DigitClassifier()
optimizer = torch.optim.Adam(dc_cnn.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

def train(model, train_loader, epochs):
    for epoch in range(epochs):
        for idx, batch in enumerate(train_loader):
            inputs, labels = batch
            preds = model(inputs)
            loss = criterion(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f'Epoch: {epoch},\tBatch: {idx},\tLoss: {loss.item()}'.expandtabs())
    
    print(f'Training complete! Final loss: {loss.item()}')

## Step 6
Train our model on the MNIST training dataloader object

In [36]:
train(dc_cnn, train_loader, epochs)

Epoch: 0,       Batch: 0,       Loss: 2.322740077972412
Epoch: 0,       Batch: 1,       Loss: 2.295907497406006
Epoch: 0,       Batch: 2,       Loss: 2.2797815799713135
Epoch: 0,       Batch: 3,       Loss: 2.27097749710083
Epoch: 0,       Batch: 4,       Loss: 2.3089494705200195
Epoch: 0,       Batch: 5,       Loss: 2.2740797996520996
Epoch: 0,       Batch: 6,       Loss: 2.3883111476898193
Epoch: 0,       Batch: 7,       Loss: 2.2660622596740723
Epoch: 0,       Batch: 8,       Loss: 2.2591471672058105
Epoch: 0,       Batch: 9,       Loss: 2.2534701824188232
Epoch: 0,       Batch: 10,      Loss: 2.2606825828552246
Epoch: 0,       Batch: 11,      Loss: 2.2664904594421387
Epoch: 0,       Batch: 12,      Loss: 2.304399251937866
Epoch: 0,       Batch: 13,      Loss: 2.341977596282959
Epoch: 0,       Batch: 14,      Loss: 2.2476322650909424
Epoch: 0,       Batch: 15,      Loss: 2.3214058876037598
Epoch: 0,       Batch: 16,      Loss: 2.241013765335083
Epoch: 0,       Batch: 17,      Loss: 

## Step 7
Save and test our model

In [34]:
torch.save(dc_cnn.state_dict(), str(time.time()))

In [45]:
def calc_accuracy(model, test_loader):
    num_correct = 0
    num_examples = len(test_loader.dataset)
    model.eval()
    for batch in test_loader:
        inputs, labels = batch
        preds = model(inputs)
        preds = torch.max(preds, axis=1) # not sure what's going on here
        preds = preds[1]
        num_correct += int(sum(preds == labels))
    
    return num_correct/num_examples * 100

In [46]:
calc_accuracy(dc_cnn, test_loader)

90.39

Achieved an accuracy of 90.39% - far better than chance. Let's see what happens with our example image.

In [65]:
display(ex_img)
preds = dc_cnn(example[0].unsqueeze(0))
pred = torch.max(preds, axis=1)[1]
print(pred)

tensor([6])


Great! It was able to pick out the number 6!